# Accessing a Python object in Scala


You can call methods and properties of Python objects in Scala code. Consider the following Python class:




In [1]:
class MyPythonClass:
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'
    
    def add(self, a, b):
        return a + b
    
    def getLength(self, thing):
        return len(thing)

pyInst = MyPythonClass()

Simple actions work in Scala:

In [3]:
pyInst.f() // note that this must be called with `()` like in Python. 

hello world

In [4]:
pyInst.i

12345

In [5]:
pyInst.add(1, Math.PI)

4.141592653589793

In [6]:
pyInst.getLength(Array(1, 2, 3))

3

Note that passing Scala values into Python only works for primitive values right now. So, for example, the follow cell will fail because Python doesn't know what to do with a Scala `Seq`. 

In [8]:
pyInst.getLength(Seq(1,2,3))

zio.FiberFailure: Fiber failed.
A checked error was not handled.
jep.JepException: <class 'TypeError'>: object of type '$colon$colon' has no len()
	at Cell1.getLength(Cell1:12)
	at jep.python.PyCallable.call(Native Method)
	at jep.python.PyCallable.callAs(PyCallable.java:107)
	at polynote.runtime.python.PythonObject$$anonfun$1.apply(PythonObject.scala:27)
	at polynote.runtime.python.PythonObject$$anonfun$1.apply(PythonObject.scala:26)
	at zio.internal.FiberContext.evaluateNow(FiberContext.scala:490)
	at zio.internal.FiberContext.zio$internal$FiberContext$$run$body$2(FiberContext.scala:776)
	at zio.internal.FiberContext$$anonfun$29.run(FiberContext.scala:776)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.java:829)

Fiber:Id(1664294554508,2863) was supposed to continue to:
  a future continuation at zio.ZIO

Note that simple Python objects like a `List` convert to their equivalents in Java (in this case, an `ArrayList`), meaning you won't get to use many Scala specific features out of the box (such as for-comprehensions to iterate over lists). To accomodate this, we created the `PythonObject`, a wrapper over Jep's `PyObject` that provides a bit of Scala sugar. 

An example is shown below where the `asScalaList` method is used to enable for-comprehension over a simple `Python` list. 

You can find the full list of supported helper methods [here](https://github.com/polynote/polynote/blob/master/polynote-runtime/src/main/scala/polynote/runtime/python/PythonObject.scala). 

In [10]:
ids = [1234, 5678, 9012, 3456]
typs = ['Number', 'Number', 'Number', 'Number']

In [11]:
case class ResultItem(id: Int, typ: String)

val resultData = for {
  i <- ids.asScalaList
  t <- typs.asScalaList
} yield ResultItem(i.as[Integer], t.as[String])

resultData 

List(ResultItem(1234,Number), ResultItem(1234,Number), ResultItem(1234,Number), ResultItem(1234,Number), ResultItem(5678,Number), ResultItem(5678,Number), ResultItem(5678,Number), ResultItem(5678,Number), ResultItem(9012,Number), ResultItem(9012,Number),…